In [28]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
csv_readin = pd.read_csv('input_data1.csv')
df = pd.DataFrame(csv_readin)

In [30]:
import tensorflow as tf
from keras import layers as klayer
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
tf.random.set_seed(1234)

print("--Get data--")
data = pd.read_csv('data_cleaned.csv',index_col=False)
data.convert_dtypes()
data.head(10)




--Get data--


,index,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays,Class,Grade_level
0,0,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,15,16,2,20,Yes,Good,Under-7,M,2
1,1,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,20,20,3,25,Yes,Good,Under-7,M,2
2,2,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,10,7,0,30,No,Bad,Above-7,L,1
3,3,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,30,25,5,35,No,Bad,Above-7,L,1
4,4,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,40,50,12,50,No,Bad,Above-7,M,2
5,5,F,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,42,30,13,70,Yes,Bad,Above-7,M,2
6,6,M,KW,KuwaIT,MiddleSchool,G-07,A,Math,F,Father,35,12,0,17,No,Bad,Above-7,L,1
7,7,M,KW,KuwaIT,MiddleSchool,G-07,A,Math,F,Father,50,10,15,22,Yes,Good,Under-7,M,2
8,8,F,KW,KuwaIT,MiddleSchool,G-07,A,Math,F,Father,12,21,16,50,Yes,Good,Under-7,M,2
9,9,F,KW,KuwaIT,MiddleSchool,G-07,B,IT,F,Father,70,80,25,70,Yes,Good,Under-7,M,2


In [31]:
#Change text data to numerical data
def convert_absenceDays (text):
	if text == 'Under-7':
		return 0 
	else:
		return 1
def convert_ParentAnsweringSurvey(text):
	if text == 'Yes':
		return 1
	else:
		return 0
def convert_ParentschoolSatisfaction(text):
	if text == 'Good':
		return 1
	else:
		return 0
def convert_StageID(text):
	if text == 'HighSchool':
		return 3
	elif text == 'MiddleSchool':
		return 2
	else:
		return 1
def convert_gender(text):
	if text == 'F':
		return 1
	else:
		return 0

In [32]:
# apply convert function to dataframe
data['absenceDays_flaged'] = data['StudentAbsenceDays'].apply(convert_absenceDays)
data['ParentAnsweringSurvey_flaged'] = data['ParentAnsweringSurvey'].apply(convert_ParentAnsweringSurvey)
data['ParentschoolSatisfaction_flaged'] = data['ParentschoolSatisfaction'].apply(convert_ParentschoolSatisfaction)
data['StageID_flaged'] = data['StageID'].apply(convert_StageID)
data['Gender_flaged'] = data['gender'].apply(convert_gender)
input_data = data.loc[:,['absenceDays_flaged','Discussion','AnnouncementsView','VisITedResources','raisedhands','ParentAnsweringSurvey_flaged','ParentschoolSatisfaction_flaged','StageID_flaged','Gender_flaged']]
true_label = data.loc[:,['Grade_level']]
x_train,x_test,y_train,y_test = train_test_split(input_data, true_label.values.ravel(),test_size=0.2,random_state=1)
x_train = np.asarray(x_train).astype(np.int32)
x_test = np.asarray(x_test).astype(np.int32)
y_train = np.asarray(y_train).astype(np.int32)
y_test = np.asarray(y_test).astype(np.int32)

In [56]:
print("--Make model--")
model = tf.keras.models.Sequential()     
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(512, activation = "relu", use_bias= True, kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(512, activation = "relu", use_bias= True))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("--Fit model--")
model.fit(x_train, y_train, epochs=22, verbose=1)
print("--Evaluate model--")
model_loss1, model_acc1 = model.evaluate(x_train,  y_train, verbose=2)
model_loss2, model_acc2 = model.evaluate(x_test,  y_test, verbose=2)
print(f"Train / Test Accuracy: {model_acc1*100:.1f}% / {model_acc2*100:.1f}%")

--Make model--
--Fit model--
Epoch 1/22
13/13 [==============================] - 1s 4ms/step - loss: 10.9574 - accuracy: 0.3844
Epoch 2/22
13/13 [==============================] - 0s 4ms/step - loss: 2.8028 - accuracy: 0.5226
Epoch 3/22
13/13 [==============================] - 0s 4ms/step - loss: 1.3033 - accuracy: 0.5025
Epoch 4/22
13/13 [==============================] - 0s 5ms/step - loss: 1.0113 - accuracy: 0.5729
Epoch 5/22
13/13 [==============================] - 0s 5ms/step - loss: 0.8721 - accuracy: 0.6055
Epoch 6/22
13/13 [==============================] - 0s 4ms/step - loss: 0.7956 - accuracy: 0.6055
Epoch 7/22
13/13 [==============================] - 0s 4ms/step - loss: 0.7427 - accuracy: 0.6533
Epoch 8/22
13/13 [==============================] - 0s 5ms/step - loss: 0.7392 - accuracy: 0.6407
Epoch 9/22
13/13 [==============================] - 0s 4ms/step - loss: 0.7053 - accuracy: 0.6583
Epoch 10/22
13/13 [==============================] - 0s 4ms/step - loss: 0.6569 - accura